# Analysing Unstructured Data

## EXERCISE 1: Working with Unstructured Data (SMS texts) - Feature Extraction


[Adapted from http://radimrehurek.com/data_science_python/]

Other references:
* http://sebastianraschka.com/Articles/2014_naive_bayes_1.html
* http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
* https://gist.github.com/zacstewart/5978000

Other options:
* http://scikit-learn.org/stable/datasets/#the-20-newsgroups-text-dataset
* http://scikit-learn.org/stable/datasets/#rcv1-dataset

### Download data from UCI ML data repo
For the following exercise, we use some real-world SMS dataset which we dynamically download from the UCI ML data repository and then store in a local 'data' directory:

In [1]:
import os
import urllib.request
import zipfile

DATA_URI = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
DATA_DIR = 'data'
ARCHIVE_NAME = 'smsspamcollection.zip'
FILE_NAME = 'SMSSpamCollection'

# set up paths (in portable, OS-agnostic way)
local_archive_path = os.path.join(DATA_DIR, ARCHIVE_NAME)
local_file_path = os.path.join(DATA_DIR, FILE_NAME)

## set up local data directory
os.makedirs(DATA_DIR, exist_ok=True)

## save file from DATA_URI to local_path
urllib.request.urlretrieve(DATA_URI, local_archive_path)

## extract content from archive
z = zipfile.ZipFile(local_archive_path, 'r')
z.extractall(DATA_DIR)
z.close()

### Read and profile data using Pandas

Pandas provides tools that streamline some of the data analysis and visualisation work we've done in previous exercises. 

[DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) is the most commonly used data structure in Pandas. It is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table.

Let's use it now to read and profile our spam data.

In [2]:
import csv
import pandas

messages = pandas.read_csv(local_file_path, sep='\t', quoting=csv.QUOTE_NONE,
                           names=["label", "message"])
print(messages)

# view aggregate statistics
messages.groupby('label').describe()

     label                                            message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5569  spam  This is the 2nd time we have tried 2 contact u...
5570   ham               Will ü b going to esplanade fr home?
5571   ham  Pity, * was in mood for that. So...any other s...
5572   ham  The guy did some bitching but I acted like i'd...
5573   ham                         Rofl. Its true to its name

[5574 rows x 2 columns]


message                                                               
        count unique                                                top freq
label                                                                       
ham      4827   4518                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

### Text feature extraction

- It's easy for humans to see the patterns that distinguish ham and spam messages. What kind of features could we feed to a machine learning algorithm?

Feeding raw text data (e.g., "Sorry, I'll call later") to a machine learning algorithm would not be very useful. As a first step let's assume that there is a systematic difference between the words used for spam and ham.

`scikit-learn` includes [several functions for creating feature vectors from text](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction). These first tokenise -- split strings into words (e.g., "Sorry", "I'll", "call", "later").

Then create feature vectors where indices correspond to a specific word and values correspond to the frequency of the corresponding word. 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# First split in to train and test sets
X_train, X_test, y_train, y_test = train_test_split(messages.message, messages.label, test_size=0.33,
                                                    random_state=5) # so we get the same results

# Fit and transform training to feature vectors of words
v = CountVectorizer(binary=True, lowercase=False)
X_train_bin = v.fit_transform(X_train)
print('Feature matrix shape:\n', X_train_bin.shape)

# scikit-learn vectorisers produce sparse array representations.
# These store only observed features for each instance instead of a complete vector.
# They're great for working with text data, but require a bit of work to inspect.

# View feature names
print('Feature names:\n', v.get_feature_names()[:10])

# View complete feature vector for row 0 as a list of words
def iter_features(X, row, names):
    for i in X[row].indices:
        yield names[i]
print('\nFeature words for row 0:\n', list(iter_features(X_train_bin, 0, v.get_feature_names())))

# Compare to original data
print('\nOriginal string for row0:\n', X_train.iloc[0])

Feature matrix shape:
 (3734, 8745)
Feature names:
 ['00', '000', '000pes', '008704050406', '0089', '0121', '01223585236', '01223585334', '0125698789', '02']

Feature words for row 0:
 ['Ok']

Original string for row0:
 Ok


C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### TODO: Feature Extraction Experimenting
Now it is your turn:
 - check the extracted features of some other rows too
 - change the CountVectorizer() to one using term counts ('binary=False') and/or ngrams ('ngram_range=(1,2)'). The lecture slides give some tips on which configuration parameters are possible. How does this affect the shape of the document term matrix and the extracted features from the rows which you investigated before?
 
Documentation of the CountVectoriser:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer

In [4]:
print('n\Original SMS string from row 10:\n', X_train.iloc[10])
print('\nFeature words for row 10:\n', list(iter_features(X_train_bin, 10, v.get_feature_names())))
print('Note how the URL had been split into separate features and how single words or digits have been ignored.')

n\Original SMS string from row 10:
 For taking part in our mobile survey yesterday! You can now have 500 texts 2 use however you wish. 2 get txts just send TXT to 80160 T&C www.txt43.com 1.50p

Feature words for row 10:
 ['you', 'get', 'in', 'to', 'can', 'www', 'For', 'taking', 'part', 'our', 'mobile', 'survey', 'yesterday', 'You', 'now', 'have', '500', 'texts', 'use', 'however', 'wish', 'txts', 'just', 'send', 'TXT', '80160', 'txt43', 'com', '50p']
Note how the URL had been split into separate features and how single words or digits have been ignored.


## *STOP PLEASE and watch next lecture video first. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.*

## EXERCISE 2: SMS spam filtering with naive Bayes

### Build pipelines and choose parameters

`scikit-learn` includes pipeline functionality that makes it possible to specify and optimise a sequence of actions.

Let's see whether the results in [Wang and Manning (2012)](http://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf) hold for the SMS spam data we're using here.

Specifically we will compare multinomial naive Bayes to support vector machine with a degree-2 polynomial kernel. We will choose the best feature representation for both:
* binary vs term frequency vs tfidf
* unigrams vs bigrams

In [9]:
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

# Pipeline for multinomial naive Bayes
mnb = Pipeline([('vect', CountVectorizer(lowercase=False)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())
               ])

# Pipeline for polynomial support vector machine
svm = Pipeline([('vect', CountVectorizer(lowercase=False)),
                ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC(C=0.1, penalty='l2'))
               ])

# Grid search parameters
param_grid = [{'vect__binary': [True],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False]
              },
              {'vect__binary': [False],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False]
              }
             ]

# Find best parameters for MNB and SVM
gs_mnb = GridSearchCV(mnb, param_grid)
gs_mnb.fit(X_train, y_train)
print('\nMNB best params:\n', gs_mnb.best_params_)

gs_svm = GridSearchCV(svm, param_grid)
gs_svm.fit(X_train, y_train)
print('\nSVM best params:\n', gs_svm.best_params_)

# Print accuracy
print('\nMNB test result:\n', classification_report(y_test, gs_mnb.predict(X_test)))
print('\nSVM test result:\n', classification_report(y_test, gs_svm.predict(X_test)))


MNB best params:
 {'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 1)}

SVM best params:
 {'tfidf__use_idf': False, 'vect__binary': False, 'vect__ngram_range': (1, 1)}

MNB test result:
               precision    recall  f1-score   support

         ham       0.95      1.00      0.98      1586
        spam       1.00      0.69      0.82       254

    accuracy                           0.96      1840
   macro avg       0.98      0.85      0.90      1840
weighted avg       0.96      0.96      0.95      1840


SVM test result:
               precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1586
        spam       0.98      0.75      0.85       254

    accuracy                           0.96      1840
   macro avg       0.97      0.88      0.92      1840
weighted avg       0.96      0.96      0.96      1840



### TODO Handling class imbalance with SVM

- Which is better? Do we care about overall performance or just one of our classes? How does this compare to Wang and Manning's result?
- The `fit()` method for `LinearSVC` includes the `class_weight` parameter with can help deal with imbalanced data. The "balanced" mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as `n_samples / (n_classes * np.bincount(y))`. Train a new SVM model with the best grid search parameters and `class_weight='balanced'`. Is this result better?
- Is there a more appropriate scoring function we could pass to `GridSearchCV` (using `sklearn.metrics.make_scorer`)? Does this give different MNB or SVM results for the parameter grid above?

In [11]:
# 1 - SVM gives a better result for spam, which is the category of interest.
# Since it's the minority category in a binary problem, 
# we should be looking at precision and recall for spam only. 

# 2 - Spam recall and f-score are better while precision is worse. 
# Which to deploy depends on the appropriate precision/recall tradeoff
svm2 = Pipeline([('vect', CountVectorizer(lowercase=False, binary=True, ngram_range=(1,1))),
                 ('tfidf', TfidfTransformer(use_idf=False)),
                 ('clf', LinearSVC(C=0.1, penalty='l2', class_weight='balanced'))
                ])
svm2.fit(X_train, y_train) 
print('\nSVM2 test result:\n', classification_report(y_test, svm2.predict(X_test)))

# 3 - We can use `f1-score` with `pos_label='spam'` and `average='binary'`
# This gives the same results for this parameter grid, but could be used to choose the best `class_weight`. 
# Note with fbeta_score, we could also specify whether we prefer precision or recall. 

from sklearn.metrics import make_scorer, f1_score
f1_scorer = make_scorer(f1_score, pos_label='spam', average='binary')
gs_mnb2 = GridSearchCV(mnb, param_grid, scoring=f1_scorer)
gs_mnb2.fit(X_train, y_train)
print('\nMNB GS2 best params:\n', gs_mnb2.best_params_)

gs_svm2 = GridSearchCV(svm, param_grid, scoring=f1_scorer)
gs_svm2.fit(X_train, y_train)
print('\nSVM GS2 best params:\n', gs_svm2.best_params_)

print('\nMNB GS2 test result:\n', classification_report(y_test, gs_mnb2.predict(X_test)))
print('\nSVM GS2 test result:\n', classification_report(y_test, gs_svm2.predict(X_test)))


SVM2 test result:
               precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1586
        spam       0.92      0.93      0.92       254

    accuracy                           0.98      1840
   macro avg       0.95      0.96      0.95      1840
weighted avg       0.98      0.98      0.98      1840


MNB GS2 best params:
 {'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 1)}

SVM GS2 best params:
 {'tfidf__use_idf': False, 'vect__binary': False, 'vect__ngram_range': (1, 1)}

MNB GS2 test result:
               precision    recall  f1-score   support

         ham       0.95      1.00      0.98      1586
        spam       1.00      0.69      0.82       254

    accuracy                           0.96      1840
   macro avg       0.98      0.85      0.90      1840
weighted avg       0.96      0.96      0.95      1840


SVM GS2 test result:
               precision    recall  f1-score   support

         ham       0.96  

## *STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.* Please watch the lecture video about Text-based Forecasting first.

## EXERCISE 3: Forecasting movie gross with support vector regression

Imagine you have an opportunity to invest in films after seeing descriptions. Perhaps you run a cinema and need to decide which films to show. If we could predict box office gross based on descriptions, then we'd have a good basis for investment decisions.

In this exercise we'll predict movie gross with support vector regression. 

### Download reviews and movie gross data

DBPedia converts Wikipedia pages into structured semantic web data. Let's use it to grab the data we need. DBpedia has a public SPARQL endpoint at http://dbpedia.org/sparql. Enter the following query and save as `CSV` under `Results Format` and click `Run Query`.

#### Query
```
PREFIX category: <http://dbpedia.org/resource/Category:>
PREFIX dbtype: <http://dbpedia.org/datatype/>
PREFIX dcterms: <http://purl.org/dc/terms/>

SELECT ?dburl ?title ?budget ?gross ?abstract
WHERE { 
 ?dburl dcterms:subject category:2013_films .

 ?dburl foaf:name ?title .
 FILTER(LANG(?title) = "en") .

 ?dburl dbo:budget ?budget .
 FILTER(xsd:float(?budget) > xsd:float("1.0E7")) .
 FILTER(datatype(?budget) = dbtype:usDollar) .

 ?dburl dbo:gross ?gross .
 FILTER(datatype(?gross) = dbtype:usDollar) .

 ?dburl dbo:abstract ?abstract .
 FILTER(LANG(?abstract) = "en") .
 FILTER(fn:string-length(?abstract) >= xsd:int(140)) .
} 
```

This will return the box office gross and the Wikipedia abstract for English films from 2013 that had a USD budget of more than 10 million. This will be our training data. Upload to your `data` directory on Jupyter Hub and rename to `movies.2013.csv`.

Run the same query with `category:2014_films` to get test data. Upload to your `data` directory on Jupyter Hub and rename to `movies.2014.csv`.

In [15]:
import csv
import os

TRAIN_NAME = 'movies.2013.csv'
TEST_NAME = 'movies.2014.csv'

def read_movies(path):
    data = []
    target = []
    for d in csv.DictReader(open(path)):
        data.append(d['abstract'])
        target.append(float(d['gross']))
    return data, target

train_path = os.path.join('data', TRAIN_NAME)
X_train, y_train = read_movies(train_path)
print(len(X_train), len(y_train))

test_path = os.path.join('data', TEST_NAME)
X_test, y_test = read_movies(test_path)
print(len(X_test), len(y_test))

FileNotFoundError: [Errno 2] No such file or directory: 'data\\movies.2013.csv'

### Select parameters for support vector regression

Let's see whether the results in [Kogan et al. (2009)](http://www.cs.cmu.edu/~nasmith/papers/kogan+levin+routledge+sagi+smith.naacl09.pdf) hold for the movie gross data we're using here.

Specifically we will choose the best feature representation for both:
* binary vs term frequency vs tfidf
* unigrams vs bigrams

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

svr = Pipeline([('vect', CountVectorizer(lowercase=True)),
                ('tfidf', TfidfTransformer()),
                ('clf', LinearSVR(epsilon=0.1, max_iter=40000))
               ])

# Grid search parameters
param_grid = {'vect__ngram_range': [(1, 1), (1, 2)],
              'vect__binary': [True, False],
              'tfidf__use_idf': [True, False],
              'tfidf__sublinear_tf': [True, False],
              'clf__C': [1e8, 1e9, 1e10],
             }

# Find best parameters
gs_svr = GridSearchCV(svr, param_grid)
gs_svr.fit(X_train, y_train)
print('Grid search mean and stdev:\n')

scoring = gs_svr.cv_results_
for mean_score, std, params in zip(scoring['mean_test_score'],scoring['std_test_score'],scoring['params']):
    print("{:0.3f} (+/-{:0.03f}) for {}".format(
            mean_score, std * 2, params))
    

print('\nSVR best params:\n', gs_svr.best_params_)
print('\nSVR r-squared on training data:\n', gs_svr.score(X_train, y_train))
print('\nSVR r-squared on test data:\n', gs_svr.score(X_test, y_test))

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_classes.py", line 499, in fit
    sample_weight=sample_weight,
  File "C:\Users\User\anaconda3\l

ValueError: could not convert string to float: 'ham'

### TODO Evaluation and discussion

- According to the 95% prediction interval, how close will our predictions be to the actual value? What if we calculate over the test data instead?
- Draw a residual plot with training data in blue and test data in green. Is a linear model appropriate for our data? Is prediction performance comparable to training performance?
- How could we improve this experimental setup? We can use `gs_svr.steps` to access feature names (`vect.get_feature_names()`) and weights (`clf.coef_`) from the pipeline components respectively. Use Python `zip` function to combine and sort these. What do the highest-weighted features tell us about our experimental setup?
- Would you use this model to pick investments?
- Maybe we're not predicting the right thing. What derived values could we predict? If you have time to try them, are they better?
- Is there other text data that might be a better predictor of box office returns?

In [ ]:
%matplotlib inline

# 1 - 
from sklearn.metrics import mean_squared_error
import math
se = math.sqrt(mean_squared_error(y_train, gs_svr.predict(X_train)))
print('Standard error:', se)
print('\nPredictions should be within {:.1f}M dollars at 95% confidence according to training data.'.format(2*se/1e6))
se_test = math.sqrt(mean_squared_error(y_test, gs_svr.predict(X_test)))
print('Predictions should be within {:.1f}M dollars at 95% confidence according to test data.'.format(2*se_test/1e6))

# 2 - Training residuals are nearly zero for most movies with a few exceptions.
#     This may indicate overfitting. The residual plot for test data is OK.
import matplotlib.pyplot as plt
print('\nResidual plot for training data (blue) and test data (green) below')
_ = plt.scatter(gs_svr.predict(X_train), y_train-gs_svr.predict(X_train), c='blue', s=40, alpha=0.5, edgecolor='white')
_ = plt.scatter(gs_svr.predict(X_test), y_test-gs_svr.predict(X_test), c='green', s=40, alpha=0.5, edgecolor='white')
_ = plt.plot([-10,50], [0,0], c='black')
_ = plt.ylabel('Residuals ($y - \hat{y}$)')
_ = plt.xlabel('Predicted values ($\hat{y}$)')

# 3 - More data!!!
#     It would be better to have another chunk of data, e.g., from 2012.
#     Then we could train on 2012 and tune our parameters on 2013.
#     2014 could then serve as a final held-out test set to evaluate generalisation.
#     It may be better to predict return / budget instead of the raw return value.
#     We should also compare to a baseline model, e.g., using budget to predict gross return.
#     Wikipedia is a bad source of descriptions for our scenario, since it is constantly
#     edited and so we have data leakage. We could use the version of the Wikipedia
#     abstract from when the movie was released.
#     Printing the highest-weighted features highlights issues of data leakage..
steps = dict(gs_svr.best_estimator_.steps) # access pipeline components directly
vect = steps['vect'] # vectorizer component
clf = steps['clf'] # classifier component
feature_weights = clf.coef_
feature_names = vect.get_feature_names()
print('\nTop 20 features:')
sorted(zip(feature_weights, feature_names), reverse=True)[:20] # print top 20 features

# 4 - No way would I deploy. The data leakage here means we can't trust this experiment setup.

# 5 - We could try the ratio of gross to budget, then multiply a prediction by
#     budget to get predicted gross.

# 6 - Reviews, e.g., from Metacritic or Rotten Tomatoes.
#     http://www.metacritic.com/feature/film-quality-vs-box-office-grosses
#     Here it would be good to compare to a baseline that predicts gross based on star ratings.